In [2]:
import sys
import numpy as np
import random
import copy

random.seed(10)

# mbnpy toolkit
sys.path.append(r"C:\Users\jb622s\git\BNS-JT\BNS_JT") 
from BNS_JT import cpm, variable

# Problem
## System
<figure>
<img src="img/house_power_prob.png" style="width:400px">
</figure>

The system consists of two power substations $S_0$ and $S_1$, eight transmission line poles $P_0$, ..., $P_7$, and seven houses $H_0$, ..., $H_6$. The substations and poles may fail being subjected to hazard scenarios $H$. Once a hazard event occurs, repair works will take place with priorities $S_0$ &rarr; $S_1$ &rarr; $P_0$ &rarr; $P_1$ &rarr; ... &rarr; $P_6$. The works are done only for failed components, and repairing a component takes a day.

Of interest is the expected number of days of power cut for each house. A house has power when the substation and poles it is connected to all survive. For example, $H_0$ has access to power if $S_0$ and $P_0$ are both in operation, while $H_1$ does if $S_0$, $P_0$, and $P_1$ are all in operation. We note that houses $H_3$ and $H_4$ have access to both substations and get power when either of the two link-sets is available. For example, $H_3$ has power either $S_0$, $P_2$, and $P_3$ work together or $S_1$ and $P_4$ are in operation.

## Bayesian network (BN) graph
The BN graph can be set up as below:
<figure>
<img src="img/power_house_bn.PNG" style="width:400px">
</figure>
 
The hazard node $H$ affects damage states of components, $S_0$, $S_1$, $P_0$, ..., $P_7$. Then, damage state of a component $X$ decides the rapair timing (i.e. how many days after a hazard occurrence) of a corresponding component, $T^X$. $T^X$ is also affected by the timing of the component right before in the repair priority, e.g. $T_1^S$ is dependent on $T^S_0$ as $S_0$ is right before $S_1$ in the repair priority. Then, actual closure time of each component, $C^X$ is decided by the corresponding damage state $X$ and $T^X$. That is, if $X$ has failed, then the closure time is the same as $T^X$, while if it is not, the closure time is zero. Finally, each house's duration of power cut, represneted by $H_0$, ..., $H_7$, is decided by the closure duration of components constituting their link-sets. For example, as aforementioned, teh first house has $S_0$ and $P_0$ in its link set, and therefore the node $H_0$ is connected to $C_0^S$ and $C^P_0$.

In the graph, random variables are represented by single border, while deterministic variables are represented by double border. 

# Modelling

## Hazard scenarios

In [3]:
cpms={}
vars={}

vars['haz'] = variable.Variable( name='haz', values=['mild','medi','inte'] ) # values=(mild, medium, intense)
cpms['haz'] = cpm.Cpm( variables=[vars['haz']], no_child=1, C=np.array([0,1,2]), p=[0.5,0.2,0.3] )

print(vars['haz'])
print(cpms['haz'])


"Variable(name=haz, B=[{0}, {1}, {2}, {0, 1}, {0, 2}, {1, 2}, {0, 1, 2}], values=['mild', 'medi', 'inte'])"
Cpm(variables=['haz'], no_child=1, C=[[0]
 [1]
 [2]], p=[[0.5]
 [0.2]
 [0.3]]


## Structural damage

In [4]:
# Substations
n_subs = 2
for i in range(n_subs):
    name = 'xs' + str(i)
    vars[name] = variable.Variable( name=name, values=['fail','surv'] ) # values=(failure, survival)
    cpms[name] = cpm.Cpm( variables = [vars[name], vars['haz']], no_child = 1,
                      C=np.array([[0,0], [1,0], [0,1], [1,1], [0,2], [1,2]]) )

cpms['xs0'].p = np.array([0.001, 0.999, 0.01, 0.99, 0.1, 0.9])
cpms['xs1'].p = np.array([0.005, 0.995, 0.05, 0.95, 0.2, 0.8])

# Poles
n_pole = 8
for i in range(n_pole):
    name = 'xp' + str(i)
    vars[name] = variable.Variable( name=name, values=['fail','surv'] ) # values=(failure, survival)
    cpms[name] = cpm.Cpm( variables = [vars[name], vars['haz']], no_child = 1,
                      C=np.array([[0,0], [1,0], [0,1], [1,1], [0,2], [1,2]]) )
    
    if i in [0,2,4,6]:
        cpms[name].p = np.array([0.001, 0.999, 0.01, 0.99, 0.1, 0.9])
    else:
        cpms[name].p = np.array([0.005, 0.995, 0.05, 0.95, 0.2, 0.8])


## Repair timing

In [6]:
max_ct = n_subs + n_pole 

# Repair priority
rep_pri = ['xs' + str(i) for i in range(n_subs)] + ['xp' + str(i) for i in range(n_pole)] # repair priority

print(rep_pri)


['xs0', 'xs1', 'xp0', 'xp1', 'xp2', 'xp3', 'xp4', 'xp5', 'xp6', 'xp7']


In [7]:
for i, x in enumerate(rep_pri):
    name = 't'+x[1:]
    vars[name] = variable.Variable( name=name, values=list(range(i+2)) )

    if i < 1: # first element
        cpms[name] = cpm.Cpm( variables = [vars[name], vars[x]], no_child = 1, C=np.array([[1,0], [0, 1]]), p=np.array([1,1]) )
    else:
        t_old_vars = vars[t_old].values

        Cx = np.empty(shape=(0,3), dtype=int)
        for v in t_old_vars:
            Cx_new = [[v, 1, v], [v+1, 0, v]]
            Cx = np.vstack( [Cx, Cx_new] )

        cpms[name] = cpm.Cpm( variables = [vars[name], vars[x], vars[t_old]], no_child = 1, C=Cx, p=np.ones(shape=(2*len(t_old_vars)), dtype=np.float32) )

    t_old = copy.deepcopy(name)


print(vars['ts0'])
print(cpms['ts0'])
print(" ")
print(vars['tp3'])
print(cpms['tp3'])

'Variable(name=ts0, B=[{0}, {1}, {0, 1}], values=[0, 1])'
Cpm(variables=['ts0', 'xs0'], no_child=1, C=[[1 0]
 [0 1]], p=[[1]
 [1]]
 
'Variable(name=tp3, B=[{0}, {1}, {2}, {3}, {4}, {5}, {6}, {0, 1}, {0, 2}, {0, 3}, {0, 4}, {0, 5}, {0, 6}, {1, 2}, {1, 3}, {1, 4}, {1, 5}, {1, 6}, {2, 3}, {2, 4}, {2, 5}, {2, 6}, {3, 4}, {3, 5}, {3, 6}, {4, 5}, {4, 6}, {5, 6}, {0, 1, 2}, {0, 1, 3}, {0, 1, 4}, {0, 1, 5}, {0, 1, 6}, {0, 2, 3}, {0, 2, 4}, {0, 2, 5}, {0, 2, 6}, {0, 3, 4}, {0, 3, 5}, {0, 3, 6}, {0, 4, 5}, {0, 4, 6}, {0, 5, 6}, {1, 2, 3}, {1, 2, 4}, {1, 2, 5}, {1, 2, 6}, {1, 3, 4}, {1, 3, 5}, {1, 3, 6}, {1, 4, 5}, {1, 4, 6}, {1, 5, 6}, {2, 3, 4}, {2, 3, 5}, {2, 3, 6}, {2, 4, 5}, {2, 4, 6}, {2, 5, 6}, {3, 4, 5}, {3, 4, 6}, {3, 5, 6}, {4, 5, 6}, {0, 1, 2, 3}, {0, 1, 2, 4}, {0, 1, 2, 5}, {0, 1, 2, 6}, {0, 1, 3, 4}, {0, 1, 3, 5}, {0, 1, 3, 6}, {0, 1, 4, 5}, {0, 1, 4, 6}, {0, 1, 5, 6}, {0, 2, 3, 4}, {0, 2, 3, 5}, {0, 2, 3, 6}, {0, 2, 4, 5}, {0, 2, 4, 6}, {0, 2, 5, 6}, {0, 3, 4, 5}, {0, 3, 4, 6}, {0, 

## Closure time

In [ ]:
#### 070324

## Power cut time

In [6]:
def get_C_max_val( vars_p ):

    def get_mv( var ): # get maximum values
        return max(var.values)
    
    vars_p.sort(key=get_mv) # sort variables with a lower maximum value (to minimise # of C's rows)

    C_new = np.empty(shape=(0,1+len(vars_p)), dtype='int32') # var: [X] + vars_p
    vals_new = []
    for i, p in enumerate(vars_p):
        vs_p = copy.deepcopy( vars_p[i].values )
        vs_p.sort()
        
        for v in vs_p:
            c_i = np.zeros(shape=(1,1+len(vars_p)), dtype='int32')
            c_i[0][i+1] = p.values.index(v)

            add = True
            for i2, p2 in enumerate(vars_p):
                if i != i2:
                    if i2 < i:
                        vs_i2 = {y for y, z in enumerate(p2.values) if z < v}

                    if i2 > i:
                        vs_i2 = {y for y, z in enumerate(p2.values) if z <= v}
                        

                    if len(vs_i2) < 1:
                        add = False
                        break
                    else:
                        st_i2 = p2.B.index(vs_i2)
                        c_i[0][i2+1] = st_i2

            if add:
                if v not in vals_new:
                    vals_new.append(v)
                c_i[0][0] = vals_new.index(v)
                C_new = np.vstack([C_new, c_i])

    return C_new, vals_new

In [7]:
# Example 1
vars_p_h0 = [vars['tp0'], vars['ts0']]
C_h0, vals_h0 = get_C_max_val(vars_p_h0)

print(C_h0, vals_h0)

# Example 2
vars_p_h1 = [vars['tp1'], vars['tp0'], vars['ts0']]
C_h1, vals_h1 = get_C_max_val(vars_p_h1)

print(C_h1, vals_h1)

[[0 0 0]
 [1 1 4]
 [1 0 1]
 [2 2 2]
 [3 2 3]] [0, 1, 2, 3]
[[ 0  0  0  0]
 [ 1  1  4  5]
 [ 1  0  1  5]
 [ 2  2  2 15]
 [ 3  2  3 25]
 [ 1  0  0  1]
 [ 2  2  4  2]
 [ 3  2 10  3]
 [ 4  2 14  4]] [0, 1, 2, 3, 4]


In [8]:
def get_C_min_val( vars_p ):

    def get_mv( var ): # get maximum values
        return max(var.values)
    
    vars_p.sort(key=get_mv) # sort variables with a lower maximum value (to minimise # of C's rows)

    C_new = np.empty(shape=(0,1+len(vars_p)), dtype='int32') # var: [X] + vars_p
    vals_new = []
    for i, p in enumerate(vars_p):
        vs_p = copy.deepcopy( vars_p[i].values )
        vs_p.sort()
        
        for v in vs_p:
            c_i = np.zeros(shape=(1,1+len(vars_p)), dtype='int32')
            c_i[0][i+1] = p.values.index(v)

            add = True
            for i2, p2 in enumerate(vars_p):
                if i != i2:
                    if i2 < i:
                        vs_i2 = {y for y, z in enumerate(p2.values) if z > v}

                    if i2 > i:
                        vs_i2 = {y for y, z in enumerate(p2.values) if z >= v}
                        

                    if len(vs_i2) < 1:
                        add = False
                        break
                    else:
                        st_i2 = p2.B.index(vs_i2)
                        c_i[0][i2+1] = st_i2

            if add:
                if v not in vals_new:
                    vals_new.append(v)
                c_i[0][0] = vals_new.index(v)
                C_new = np.vstack([C_new, c_i])

    return C_new, vals_new

In [9]:
# Example 1
vars_p_h0 = [vars['tp0'], vars['ts0']]
C_h0, vals_h0 = get_C_min_val(vars_p_h0)

print(C_h0, vals_h0)

# Example 2
vars_p_h1 = [vars['tp1'], vars['tp0'], vars['ts0']]
C_h1, vals_h1 = get_C_min_val(vars_p_h1)

print(C_h1, vals_h1)

[[ 0  0 14]
 [ 1  1 13]
 [ 0  1  0]] [0, 1]
[[ 0  0 14 30]
 [ 1  1 13 29]
 [ 0  1  0 30]
 [ 0  1 13  0]] [0, 1]


In [10]:
cut_lin = {} # cut-set of link-sets for each house

cut_lin['h0'] = [['ts0', 'tp0']]
cut_lin['h1'] = [['ts0', 'tp0', 'tp1']]
cut_lin['h2'] = [['ts0', 'tp2']]
cut_lin['h3'] = [['ts0', 'tp2', 'tp3'], ['ts1','tp4']]
cut_lin['h4'] = [['ts0', 'tp2', 'tp3', 'tp5'], ['ts1', 'tp4','tp5']]
cut_lin['h5'] = [['ts1','tp6']]
cut_lin['h6'] = [['ts1','tp6', 'tp7']]

for h, sets in cut_lin.items():
    if len(sets) == 1:              
        vars_h = [vars[x] for x in sets[0]]

        C_h, vals_h = get_C_max_val( vars_h )
        vars[h] = variable.Variable( name=h, values=vals_h )
        cpms[h] = cpm.Cpm( variables=[vars[h]] + vars_h, no_child=1, C=C_h, p=np.ones(shape=(len(C_h),1), dtype='float64') )

    else:
        names_hs = [h+str(i) for i in range(len(sets))]
        for h_i, s_i in zip(names_hs, sets):
            vars_h_i = [vars[x] for x in s_i]

            C_h_i, vals_h_i = get_C_max_val( vars_h_i )
            vars[h_i] = variable.Variable( name=h_i, values=vals_h_i )
            cpms[h_i] = cpm.Cpm( variables=[vars[h_i]] + vars_h_i, no_child=1, C=C_h_i, p=np.ones(shape=(len(C_h_i),1), dtype='float64') )

        vars_hs = [vars[n] for n in names_hs]
        C_h, vals_h = get_C_min_val( vars_hs )
        vars[h] = variable.Variable( name=h, values=vals_h )
        cpms[h] = cpm.Cpm( variables=[vars[h]] + vars_hs, no_child=1, C=C_h, p=np.ones(shape=(len(C_h),1), dtype='float64') )


In [11]:
print(vars['h1'])
print(cpms['h1'])

print(vars['h3'])
print(cpms['h3'])

print(vars['h30'])
print(cpms['h30'])

'Variable(name=h1, B=[{0}, {1}, {2}, {3}, {4}, {0, 1}, {0, 2}, {0, 3}, {0, 4}, {1, 2}, {1, 3}, {1, 4}, {2, 3}, {2, 4}, {3, 4}, {0, 1, 2}, {0, 1, 3}, {0, 1, 4}, {0, 2, 3}, {0, 2, 4}, {0, 3, 4}, {1, 2, 3}, {1, 2, 4}, {1, 3, 4}, {2, 3, 4}, {0, 1, 2, 3}, {0, 1, 2, 4}, {0, 1, 3, 4}, {0, 2, 3, 4}, {1, 2, 3, 4}, {0, 1, 2, 3, 4}], values=[0, 1, 2, 3, 4])'
Cpm(variables=['h1', 'ts0', 'tp0', 'tp1'], no_child=1, C=[[ 0  0  0  0]
 [ 1  1  4  5]
 [ 1  0  1  5]
 [ 2  2  2 15]
 [ 3  2  3 25]
 [ 1  0  0  1]
 [ 2  2  4  2]
 [ 3  2 10  3]
 [ 4  2 14  4]], p=[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
'Variable(name=h3, B=[{0}, {1}, {2}, {3}, {4}, {5}, {6}, {0, 1}, {0, 2}, {0, 3}, {0, 4}, {0, 5}, {0, 6}, {1, 2}, {1, 3}, {1, 4}, {1, 5}, {1, 6}, {2, 3}, {2, 4}, {2, 5}, {2, 6}, {3, 4}, {3, 5}, {3, 6}, {4, 5}, {4, 6}, {5, 6}, {0, 1, 2}, {0, 1, 3}, {0, 1, 4}, {0, 1, 5}, {0, 1, 6}, {0, 2, 3}, {0, 2, 4}, {0, 2, 5}, {0, 2, 6}, {0, 3, 4}, {0, 3, 5}, {0, 3, 6}, {0, 4, 5}, {0, 4, 6}, {0, 5, 6}, {1, 2, 3}

# Inference

In [12]:
def get_inf_vars( vars_star, cpms, VE_ord = None ):

    """
    INPUT:
    - vars_star: a list of variable names, whose marginal distributions are of interest
    - cpms: a list of CPMs
    - VE_ord (optional): a list of variable names, representing a VE order. The output list of vars_inf is sorted accordingly.
    OUPUT:
    - vars_inf: a list of variable names
    """

    vars_inf = [] # relevant variables for inference
    vars_inf_new = vars_star
    while len(vars_inf_new) > 0:
        v1 = copy.deepcopy( vars_inf_new[0] )
        vars_inf_new.remove(v1)
        vars_inf.append(v1)

        v1_sco = [x.name for x in cpms[v1].variables] # Scope of v1
        for p in v1_sco:
            if p not in vars_inf and p not in vars_inf_new:
                vars_inf_new.append(p)

    if VE_ord is not None:
        def get_ord_inf( x, VE_ord ):
            return VE_ord.index(x) 
        
        vars_inf.sort( key=(lambda x: get_ord_inf(x, VE_ord)) )

    return vars_inf

In [13]:
n_hou = len(cut_lin)

VE_ord= ['haz'] + ['xs'+str(i) for i in range(n_subs)] + ['xp'+str(i) for i in range(n_pole)] + \
        ['ts'+str(i) for i in range(n_subs)] + ['tp'+str(i) for i in range(n_pole)] + \
        ['h'+str(i) for i in range(n_hou)]

# Example 1
h_star1 = ['h0']
vars_inf1 = get_inf_vars( h_star1, cpms, VE_ord )
print(vars_inf1)

# Example 2
h_star2 = ['h0','h1']
vars_inf2 = get_inf_vars( h_star2, cpms, VE_ord )
print(vars_inf2)

['haz', 'xs0', 'xs1', 'xp0', 'ts0', 'ts1', 'tp0', 'h0']
['haz', 'xs0', 'xs1', 'xp0', 'xp1', 'ts0', 'ts1', 'tp0', 'tp1', 'h0', 'h1']


In [34]:
def merge_cpms( cpm1, cpm2 ):
    # cpm1 and cpm2 must have the same scope.

    M_new = copy.deepcopy( cpm1 )
    C1_list = cpm1.C.tolist()
    for c1, p1 in zip( cpm2.C, cpm2.p ):
        if c1 in C1_list:
            idx = C1_list.index(c1)
            M_new.p[idx] += p1
        else:
            M_new.C = np.vstack( (M_new.C, c1) )
            M_new.p = np.vstack( (M_new.p, p1) )

    return M_new

In [54]:
def cal_Msys_by_cond_VE( cpms, vars, cond_names, ve_names, sys_name ):
    """
    INPUT:
    - cpms: a dictionary of cpms
    - vars: a dictionary of variables
    - cond_names: a list of variables to be conditioned
    - oth_names: a list of variables to be eliminated by VE
    - sys_name: a system variable's name (NB not list!) **FUTHER RESEARCH REQUIRED: there is no efficient way yet to compute a joint distribution of more than one system event

    OUTPUT:
    - Msys: a cpm containing the marginal distribution of variable 'sys_name'
    """

    ve_vars = [vars[v] for v in ve_names if v != sys_name] # other variables

    cpms_inf = {v: cpms[v] for v in ve_names}
    cpms_inf[sys_name] = cpms[sys_name]
    cond_cpms = [cpms[v] for v in cond_names]

    M_cond = cpm.prod_cpms( cond_cpms )
    n_crows = len(M_cond.C)

    for i in range(n_crows):
        m1 = M_cond.get_subset([i])
        VE_cpms_m1 = cpm.condition( cpms_inf, m1.variables, m1.C[0] )

        m_m1 = cpm.variable_elim( VE_cpms_m1, ve_vars )
        m_m1 = m_m1.product(m1)
        m_m1 = m_m1.sum(cond_names)

        if i < 1:
            Msys = copy.deepcopy( m_m1 )
        else:
            Msys = merge_cpms( Msys, m_m1 )
    
    return Msys
    

In [61]:
cond_names = ['haz']

# H0
vars_inf_h0 = get_inf_vars( ['h0'], cpms, VE_ord )
ve_names = [x for x in vars_inf_h0 if x not in cond_names]

Msys_h0 = cal_Msys_by_cond_VE( cpms, vars, cond_names, ve_names, 'h0' )
print(Msys_h0)
print(sum(Msys_h0.p))

# H7
vars_inf_h6 = get_inf_vars( ['h6'], cpms, VE_ord )
ve_names = [x for x in vars_inf_h6 if x not in cond_names]

Msys_h6 = cal_Msys_by_cond_VE( cpms, vars, cond_names, ve_names, 'h6' )
print(Msys_h6)
print(sum(Msys_h6.p))

Cpm(variables=['h0'], no_child=1, C=[[0]
 [1]
 [2]
 [3]], p=[[8.77124498e-01]
 [1.08852008e-01]
 [1.34224925e-02]
 [6.01002500e-04]]
[1.]
Cpm(variables=['h6'], no_child=1, C=[[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]], p=[[6.90410249e-01]
 [1.65589430e-01]
 [9.01282113e-02]
 [3.94423861e-02]
 [1.17121852e-02]
 [2.36083768e-03]
 [3.24764791e-04]
 [3.00769304e-05]
 [1.79520093e-06]
 [6.24000037e-08]
 [9.60000006e-10]]
[1.]
